### --- Day 10: Syntax Scoring ---
You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

Syntax error in navigation subsystem on line: all of them
All of them?! The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

If a chunk opens with (, it must close with ).
If a chunk opens with [, it must close with ].
If a chunk opens with {, it must close with }.
If a chunk opens with <, it must close with >.
So, () is a legal chunk that contains no other chunks, as is []. More complex but valid chunks include ([]), {()()()}, <([{}])>, [<>({}){}[([])<>]], and even (((((((((()))))))))).

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

For example, consider the following navigation subsystem:

    [({(<(())[]>[[{[]{<()<>>
    [(()[<>])]({[<{<<[]>>(
    {([(<{}[<>[]}>{[]{[(<()>
    (((({<>}<{<{<>}{[]{[]{}
    [[<[([]))<([[{}[[()]]]
    [{[{({}]{}}([{[{{{}}([]
    {<[[]]>}<{[{[{[]{()[[[]
    [<(<(<(<{}))><([]([]()
    <{([([[(<>()){}]>(<<{{
    <{([{{}}[<[[[<>{}]]]>[]]
Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

    {([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
    [[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
    [{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
    [<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
    <{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.
Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

): 3 points.
]: 57 points.
}: 1197 points.
>: 25137 points.
In the above example, an illegal ) was found twice (2*3 = 6 points), an illegal ] was found once (57 points), an illegal } was found once (1197 points), and an illegal > was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

### --- Part Two ---
Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In the example above, there are five incomplete lines:

    [({(<(())[]>[[{[]{<()<>> - Complete by adding }}]])})].
    [(()[<>])]({[<{<<[]>>( - Complete by adding )}>]}).
    (((({<>}<{<{<>}{[]{[]{} - Complete by adding }}>}>)))).
    {<[[]]>}<{[{[{[]{()[[[] - Complete by adding ]]}}]}]}>.
    <{([{{}}[<[[[<>{}]]]>[]] - Complete by adding ])}>.
Did you know that autocomplete tools also have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

    ): 1 point.
    ]: 2 points.
    }: 3 points.
    >: 4 points.
So, the last completion string above - ])}> - would be scored as follows:

Start with a total score of 0.
Multiply the total score by 5 to get 0, then add the value of ] (2) to get a new total score of 2.
Multiply the total score by 5 to get 10, then add the value of ) (1) to get a new total score of 11.
Multiply the total score by 5 to get 55, then add the value of } (3) to get a new total score of 58.
Multiply the total score by 5 to get 290, then add the value of > (4) to get a new total score of 294.
The five lines' completion strings have total scores as follows:

    }}]])})] - 288957 total points.
    )}>]}) - 5566 total points.
    }}>}>)))) - 1480781 total points.
    ]]}}]}]}> - 995444 total points.
    ])}> - 294 total points.
Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [105]:
def syntax_input(file:str) -> list:
    with open(file, 'r') as file:
        return [l.strip() for l in file]


def check_syntax(input_line:str) -> str:
    """Check whether closing brackets match.  Return incorrect closing bracket"""
    opening = ['(','[','{','<']
    closing = [')',']','}','>']
    current = []
    for char in input_line:
        if char in opening:
            current.append(char)
        elif char in closing and char == closing[opening.index(current[-1])]:
            current.pop(-1)
        elif char in closing and char != closing[opening.index(current[-1])]:
            return char
    
def check_file(input_lines:list) -> list:
    """
    Iterate throughout list of strings. Check whether closing brackets match.
    Return list of first incorrect closing bracket per line.
    """
    errors = []
    for l in input_lines:
        errors.append(check_syntax(l))
    return errors
    
def calc_error_score(errors:list) -> int:
    """
    Return score of all incorrect closing brackets
    """
    score = 0
    for e in errors:
        score += 3 if e == ')' else 0
        score += 57 if e == ']' else 0
        score += 1197 if e == '}' else 0
        score += 25137 if e == '>' else 0
    return score


def fixable_syntax_score(input_line:str) -> str:
    """
    Check single line.  If no incorrect closing brackets but if not complete, 
    determine score of brackets required to properly close in correct order.
    
    If incorrect closing bracket found, then pass.
    """
    score = 0
    opening = ['(','[','{','<']
    closing = [')',']','}','>']
    current = []
    line_length = len(input_line)
    for i, char in enumerate(input_line):
        if char in opening:
            current.append(char)
        elif char in closing and char == closing[opening.index(current[-1])]:
            current.pop(-1)
        elif char in closing and char != closing[opening.index(current[-1])]:
            return None
        # Now check it's the last line
        if i == (len(input_line) - 1):
            for x in reversed(current):
                input_line += closing[opening.index(x)]
                score = (score * 5 + 1) if x == '(' else score
                score = (score * 5 + 2) if x == '[' else score
                score = (score * 5 + 3) if x == '{' else score
                score = (score * 5 + 4) if x == '<' else score
    return score

def fix_file_score(input_lines:list) -> list:
    """
    Check all lines in list, determine middle score of all fixable lines
    """
    scores = []
    for l in input_lines:
        score = fixable_syntax_score(l)
        if score is not None:
            scores.append(score)
    scores.sort()
    middle_idx = len(scores)//2
    return scores[middle_idx]

In [107]:
# Input
syntax = syntax_input('data/d10_syntax.txt')

# Pt 1
# Parsing errors
errors = check_file(syntax)
score = calc_error_score(errors)
print(f'Pt 1: The total error score is: {score}')

# Pt 2
# Fixable lines score
fixable_scores = fix_file_score(syntax)
print(f'Pt 2: The total fixable syntax score is: {fixable_scores}')

The total error score is: 288291
The total fixable syntax score is: 820045242
